In [ ]:
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')   # Head-less 설정
options.add_argument('--no-sandbox') # 보안 기능 비활성화 (샌드박스라는 공간을 비활성화 시킨다는 뜻)
options.add_argument('--disable-dev-shm-usage') # /dev/shm(공유메모리) 디렉토리를 사용하지 않는다는 뜻
browser = webdriver.Chrome('chromedriver', options=options)

In [ ]:
#해당 url로 이동
url = "https://www.naver.com/"
browser.get(url)

In [ ]:
# (1) 검색창 클릭
# (2) 검색창에 단어(날씨)를 입력 => .send_keys()
# (3) 검색창 아이콘 클릭
browser.find_element(By.ID, 'query').click()
browser.find_element(By.ID, 'query').send_keys("날씨")
browser.find_element(By.ID, 'search_btn').click()

In [ ]:
url = "https://www.google.com"
browser.get(url)
browser.find_element(By.CLASS_NAME, 'RNmpXc')

In [ ]:
url = 'https://weather.naver.com/'
browser.get(url)

before_temp = browser.find_element(By.CLASS_NAME, 'current').text

before_temp.split("\n")[1][:-1] # [현재온도, 11.1]

In [ ]:
# 습도 체감 동풍 - 변수로 따로 담아주세요.
data = browser.find_element(By.CLASS_NAME, 'weather_table').text
data = data.split(" ")

hum = data[1]
temp = data[3]
wind = data[5]

print(hum, temp, wind)

In [ ]:
parent = browser.find_element(By.CLASS_NAME, 'weather_table')

# child = parent.find_element(By.TAG_NAME, 'li')
child_list = parent.find_elements(By.TAG_NAME, 'li')

for child in child_list:
  print(child.text)

### 네이버 증권
- 코스피, 코스닥 데이터 가져오기

In [ ]:
url = 'https://finance.naver.com/'
browser.get(url)

In [ ]:
datas = browser.find_elements(By.CLASS_NAME, 'dn')

for elem in datas:
  print(elem.text, "\n") # 예약어. 이미 파이썬 언어가 사용 중인 변수값.

In [ ]:
parent = browser.find_element(By.ID, '_topItems1')

childs = parent.find_elements(By.TAG_NAME, 'tr')

for child in childs:
  print(child.text, "\n")

### 구글 뉴스 크롤링

In [ ]:
# 1단계 : 1건의 뉴스 데이터 가져오기 (단건)
# - 뉴스제목 / 뉴스 링크 / 신문사
title = browser.find_element(By.CLASS_NAME, 'ipQwMb').text
link = browser.find_element(By.CLASS_NAME, 'VDXfz').get_attribute('href')
company = browser.find_element(By.CLASS_NAME, 'lqNvvd').get_attribute('alt')

print(title, link, company)

In [ ]:
# 2단계 : 12건의 뉴스 데이터 다 가져오기 (다건)
# - 뉴스제목 / 뉴스 링크 / 신문사 / 작성시간 / 이미지링크

titles = browser.find_elements(By.CLASS_NAME, 'ipQwMb')
title_list = []
for i in titles:
  title_list.append(i.text)

# print(title_list)

links = browser.find_elements(By.CLASS_NAME, 'VDXfz')
link_list = []
for i in links:
  link_list.append(i.get_attribute('href'))

print(link_list)

In [ ]:
url = 'https://e.kakao.com/search?q=%EC%B6%98%EC%8B%9D%EC%9D%B4'
browser.get(url)

In [ ]:
browser.find_element(By.CLASS_NAME, 'img_default').get_attribute('src')

In [ ]:
# 이미지 링크
imgs = browser.find_elements(By.CLASS_NAME, 'img_hover')
img_list = []
for i in imgs:
  img_list.append(i.get_attribute('src'))

# 파일명
titles = browser.find_elements(By.CLASS_NAME, 'txt_tit')
title_list = []
for i in titles:
  title_list.append(i.text)

import urllib.request
for i in range(10):
  urllib.request.urlretrieve(img_list[i], title_list[i]+".png")